In [17]:
import sqlite3
import pandas as pd

CURRENCY_PAIRS = ["USDSEK", "CHFHKD", "AUDCHF","EURCHF","CADCHF"]

def first(pair):
    conn = sqlite3.connect('dataset.db')
    # Load data into a pandas dataframe
    df = pd.read_csv(f'/Users/jason/dataset1/{pair}.csv')
    cursor = conn.cursor()
    # Write the data to a new table in the SQLite database
    df.to_sql(f'{pair}', conn, if_exists='replace', index=False)
    # Execute SQL queries on the database
    cursor = conn.cursor()
    # Hourly basis
    cursor.execute(f"""
    SELECT strftime('%Y-%m-%d %H:00:00', datetime) as Hour, AVG(vw) as Avg_VWAP, SUM(n) as Total_Transactions
    FROM {pair}
    GROUP BY Hour
    """)
    print(cursor.fetchall())
    # Same starting date
    cursor.execute(f"""
    SELECT *
    FROM {pair}
    WHERE datetime >= '2011-01-01 00:00:00'
    """)
    print(cursor.fetchall())
    # Same ending date
    cursor.execute(f"""
    SELECT *
    FROM {pair}
    WHERE datetime <= '2011-12-31 23:59:59'
    """)
    print(cursor.fetchall())
    # Hourly price (VWAP)
    cursor.execute(f"""
    SELECT strftime('%Y-%m-%d %H:00:00', datetime) as Hour, AVG(vw) as Avg_VWAP
    FROM {pair}
    GROUP BY Hour
    """)
    print(cursor.fetchall())
    # Hourly liquidity (number of transactions)
    cursor.execute(f"""
    SELECT 
    strftime('%Y-%m-%d %H:00:00', datetime) AS hour,
    SUM(vw * v) / SUM(v) AS hourly_vwap,
    SUM(v) AS hourly_liquidity
    FROM {pair}
    WHERE datetime BETWEEN '2011-01-01 00:00:00' AND '2011-12-31 23:59:59'
    GROUP BY hour
    """)
    print(cursor.fetchall())
    # Close the connection to the database
    conn.close()

def second(pair):
    conn = sqlite3.connect('dataset.db')
    # read data from the database into a pandas dataframe
    df1 = pd.read_sql_query(f"SELECT * FROM {pair}", conn)

    # convert the datetime column to a pandas datetime object
    df1['datetime'] = pd.to_datetime(df1['datetime'])

    # set the datetime column as the index
    df1.set_index('datetime', inplace=True)

    # resample the dataframe on an hourly basis
    hourly_df = df1.resample('H').agg({'v': 'sum', 
                                   'vw': 'mean',
                                   'o': 'first',
                                   'c': 'last',
                                   'h': 'max',
                                   'l': 'min',
                                   't': 'sum',
                                   'n': 'sum'})

    # calculate the hourly VWAP and hourly liquidity
    hourly_df['hourly_vwap'] = hourly_df['vw'] * hourly_df['v']
    hourly_df['hourly_liquidity'] = hourly_df['n']

    # insert the hourly VWAP and hourly liquidity back into the database
    hourly_df.to_sql(f'{pair}_hourly_data', conn, if_exists='replace')

    conn.close()

def third(pair):
    conn = sqlite3.connect('dataset.db')
    # read data from the database into a pandas dataframe
    df2 = pd.read_sql_query(f"SELECT * FROM {pair}_hourly_data", conn)
    
    # convert the datetime column to a pandas datetime object
    df2['datetime'] = pd.to_datetime(df2['datetime'])

    # set the datetime column as the index
    df2.set_index('datetime', inplace=True)

    # resample the dataframe on an hourly basis
    six_hourly_df = df2.resample('6H').agg({'v': 'mean',
                                'vw': 'mean',
                                'o': 'mean',
                                'c': 'mean',
                                'h': 'max',
                                'l': 'min',
                                't': 'mean',
                                'n': 'mean',
                                'hourly_vwap': 'mean',
                                'hourly_liquidity': 'mean' })

    # calculate fd
    six_hourly_df['fd'] = six_hourly_df['n'] / (six_hourly_df['h'] - six_hourly_df['l'])

    # insert the hourly VWAP and hourly liquidity back into the database
    six_hourly_df.to_sql(f'{pair}_six_hourly_data', conn, if_exists='replace')
    conn.close()

In [18]:
def main():
    for pair in CURRENCY_PAIRS:
        first(pair)
        second(pair)
        third(pair)
    

In [19]:
if __name__ == "__main__":
    main()

[(None, 6.491406940758405, 25684351)]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(None, 8.808090852689736, 6605102)]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(None, 0.9137804578933559, 50693947)]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(None, 1.2327545270448421, 42988806)]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(None, 1.0120248570840864, 21450981)]
[(360374, '63abc4cfb405edb0add7478f', 4, 0.9387, 0.93875, 0.93875, 0.93875, 0.9387, 1293990000000.0, 4, '2011/1/2 17:00')]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

